In [ ]:
sc.stop()

In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

### Семинар

In [ ]:
sc.getConf().getAll()

In [ ]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [ ]:
rdd = sc.parallelize(collection)

In [ ]:
rdd

In [ ]:
rdd.count()

In [ ]:
rdd.take(10)

In [ ]:
rdd.getNumPartitions()

### Загружаем внешний датасет

In [ ]:
!hdfs dfs -ls /lectures/lecture01/data/ips.txt

In [ ]:
rdd2 = sc.textFile("/lectures/lecture01/data/ips.txt")

In [ ]:
rdd2

In [ ]:
rdd2.take(10)

In [ ]:
rdd2.count()

In [ ]:
# в наших данных 2 партиции
rdd2.getNumPartitions()

In [ ]:
# меняем кол-во партиций в наших данных, чтобы использовались все executors (все были зайдействованы)
rdd2.repartition(3).saveAsTextFile("ips_3_repart.txt")

In [ ]:
rdd3 = sc.textFile("ips_3_repart.txt")

In [ ]:
rdd3

In [ ]:
rdd3.getNumPartitions()

In [ ]:
rdd = sc.parallelize(range(1000000))
rdd2 = rdd.filter(lambda x: x % 2)

In [ ]:
rdd2.take(5)

In [ ]:
# трансформация RDD не приводит к изменению кол-ва партиций
rdd2.getNumPartitions()

In [ ]:
rdd2.count()

In [ ]:
rdd = sc.parallelize(range(1000000))

rdd2 = rdd.filter(lambda x: x % 1000).filter(lambda x: x % 7)
rdd3 = rdd2.map(lambda x: x * 2)
# собрать рез-ты вычислений на драйвере
rdd3.collect()

In [ ]:
rdd.collect()[:20]

In [ ]:
rdd.takeSample(withReplacement=False, num=20, seed=5757)

In [ ]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")
rdd.take(5)

In [ ]:
ips = rdd.map(lambda x: x.split("\t"))
ips.take(5)

In [ ]:
# отфильтровали записи
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")
ips_filtered.take(5)

In [ ]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")
raw_logs.getNumPartitions()

In [ ]:
raw_logs.take(5)

In [ ]:
logs = raw_logs.map(lambda x: x.split("\t"))
logs.take(5)

In [ ]:
type(logs)

In [ ]:
# разделяем 3-ий элемент в каждом подмассиве
logs.flatMap(lambda x: x[2].split()).take(20)

In [ ]:
words = logs.flatMap(lambda x: x[2].split())
words.take(5)

In [ ]:
# считаем кол-во уникальных слов
words.groupBy(lambda x: x).count()

In [ ]:
words.groupBy(lambda x: x).take(20)

In [ ]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")
ips = rdd.map(lambda x: x.split("\t"))
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")
ips_filtered.take(5)

In [ ]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")
logs = raw_logs.map(lambda x: x.split("\t"))
words = logs.flatMap(lambda x: x[2].split())
words.groupBy(lambda x: x).count()

In [ ]:
%%time
ips.count()

In [ ]:
%%time
ips.top(10)

In [ ]:
ips_cached = ips.cache()

In [ ]:
%%time
ips_cached.count()

In [ ]:
%%time
ips_cached.top(20)

In [ ]:
# удаление данных из кэша
ips_cached.unpersist()

### Лаба

In [ ]:
!hdfs dfs -ls /labs/laba01/ml-100k

In [ ]:
# список всех фильмов
rdd_hw1 = sc.textFile('/labs/laba01/ml-100k/u.item')
rdd_hw1

In [ ]:
rdd_hw1.take(5)

In [ ]:
rdd_hw1.map(lambda x: x.split("|")).take(5)

In [ ]:
# кол-во фильмов
rdd_hw1.map(lambda x: x.split("|")).flatMap(lambda x: x[0].split()).groupBy(lambda x: x).count()

In [ ]:
rdd_hw11 = rdd_hw1.map(lambda x: x.split("|"))
rdd_hw11.take(96)[-1] #.drop(96)

In [ ]:
# оценки фильмов
rdd_hw2 = sc.textFile('/labs/laba01/ml-100k/u.data')
rdd_hw2

In [ ]:
rdd_hw2.take(5)

In [ ]:
rdd_hw21 = rdd_hw2.map(lambda x: x.split("\t"))
rdd_hw21.take(5)

In [ ]:
c1 = rdd_hw21.flatMap(lambda x: x[0].split())
c1.take(5)

In [ ]:
c1.groupBy(lambda x: x).count()

In [ ]:
c2 = rdd_hw21.flatMap(lambda x: x[1].split())
c2.take(5)

In [ ]:
c2.groupBy(lambda x: x).count()

In [ ]:
rdd_hw21.count()

In [ ]:
rdd_hw3 = sc.textFile('/labs/laba01/ml-100k/u.user')
rdd_hw3

In [ ]:
rdd_hw3.take(5)

In [ ]:
rdd_hw4 = sc.textFile('/labs/laba01/ml-100k/u.info')
rdd_hw4.take(5)

In [ ]:
# отбираем 96-ый фильм
my_film_rdd = rdd_hw21.filter(lambda x: x[1] == '96')
my_film_rdd.take(5)

In [ ]:
# сколько оценок моего фильма
my_film_rdd.count()

In [ ]:
my_film_rdd.groupBy(lambda x: x[2]).count()

In [ ]:
c3 = my_film_rdd.flatMap(lambda x: x[2].split())
marks_count = dict(c3.countByValue())

In [ ]:
marks_count

In [ ]:
len(marks_count)

In [ ]:
hist_film = []
for i in range(1, len(marks_count)+1):
    hist_film.append(marks_count[str(i)])
hist_film 

In [ ]:
# rdd_hw21
c3_all = rdd_hw21.flatMap(lambda x: x[2].split())
marks_count_all = dict(c3_all.countByValue())
marks_count_all

In [ ]:
hist_all = []
for i in range(1, len(marks_count_all)+1):
    hist_all.append(marks_count_all[str(i)])
hist_all

In [ ]:
lab01 = {}
lab01['hist_film'] = hist_film
lab01['hist_all'] = hist_all
lab01

In [ ]:
import json

In [ ]:
with open('lab01.json', 'w') as f:
    json.dump(lab01, f)